In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from transformers import AutoTokenizer, AutoModel, AutoModelForTokenClassification, Trainer, TrainingArguments, GPT2Tokenizer, GPT2LMHeadModel
import spacy
import torch
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re

In [ ]:
# Load LLM for embeddings (BERT)
tokenizers = AutoTokenizer.from_pretrained('bert-base-uncased')
embedding_model = AutoModel.from_pretrained('bert-base-uncased')

# Preprocess query to get embeddings
def preprocess_query(query):
    inputs = tokenizers(query, return_tensors='pt')
    outputs = embedding_model(**inputs)
    return outputs.last_hidden_state

In [ ]:
data = pd.read_excel('Intent_training_data.xlsx')
# data
df = pd.DataFrame(data)
y_train_intent = df['Label']
X_train = df['Intent_Data']

In [ ]:
# Generating embeddings for training
X_train_embeddings = [preprocess_query(query).mean(dim=1).detach().numpy()[0] for query in X_train]

# Training the intent recognition model
intent_model = SVC(probability=True)
intent_model.fit(X_train_embeddings, y_train_intent)

print(len(X_train_embeddings), len(y_train_intent))
print(X_train[249],y_train_intent[249], y_train_intent[498])

500 500
"Fetch information on HP hardware in Section D", 0 1


In [ ]:
# Intent recognition using the trained SVC
def recognize_intent(query, model):
    embedding = preprocess_query(query).mean(dim=1).detach().numpy()[0]
    intent_proba = model.predict_proba([embedding])
    best_intent_idx = intent_proba.argmax()
    best_intent_confidence = intent_proba[0][best_intent_idx]

    intent = model.classes_[best_intent_idx]
    return intent, best_intent_confidence


In [ ]:
# Token Classification  training
# Training examples for Equipment ID
Eq_ID = [

    "NOP/LGS/STC/01", "Equipment_ID_NOP/LGS/STC/01", "ID_NOP/LGS/STC/01", "STC/01",
    "NOP/LGS/STC/02", "Equipment_ID_NOP/LGS/STC/02", "ID_NOP/LGS/STC/02", "STC/02",
    "NOP/LGS/STC/03", "Equipment_ID_NOP/LGS/STC/03", "ID_NOP/LGS/STC/03", "STC/03",
    "NOP/LGS/STC/04", "Equipment_ID_NOP/LGS/STC/04", "ID_NOP/LGS/STC/04", "STC/04",
    "NOP/LGS/STC/05", "Equipment_ID_NOP/LGS/STC/05", "ID_NOP/LGS/STC/05", "STC/05",
    "NOP/LGS/STC/06", "Equipment_ID_NOP/LGS/STC/06", "ID_NOP/LGS/STC/06", "STC/06",
    "NOP/LGS/NPRN/01", "Equipment_ID_NOP/LGS/NPRN/01", "ID_NOP/LGS/NPRN/01", "NPRN/01",
    "NOP/LGS/KVM/01", "Equipment_ID_NOP/LGS/KVM/01", "ID_NOP/LGS/KVM/01", "KVM/01",
    "NOP/LGS/SW/01", "Equipment_ID_NOP/LGS/SW/01", "ID_NOP/LGS/SW/01", "SW/01",
    "NOP/LGS/GEI/03", "Equipment_ID_NOP/LGS/GEI/03", "ID_NOP/LGS/GEI/03", "GEI/03",
    "NOP/LGS/UHF/RX", "Equipment_ID_NOP/LGS/UHF/RX", "ID_NOP/LGS/UHF/RX", "UHF/RX",
    "NOP/LGS/VHF/TX", "Equipment_ID_NOP/LGS/VHF/TX", "ID_NOP/LGS/VHF/TX", "VHF/TX",
    "LAN/SW/01", "Equipment_ID_LAN/SW/01", "ID_LAN/SW/01", "SW/01",
    "SCC_LAN/SW/01", "Equipment_ID_SCC_LAN/SW/01", "ID_SCC_LAN/SW/01", "SW/01",
    "RKVM/01", "Equipment_ID_RKVM/01", "ID_RKVM/01", "/01",
    "STATION_GW1", "Equipment_ID_STATION_GW1", "ID_STATION_GW1", "GW1",
    "NOP/LGS/TH_COM/01", "Equipment_ID_NOP/LGS/TH_COM/01", "ID_NOP/LGS/TH_COM/01", "TH_COM/01",
    "Replacement_for_STC", "Replacement_STC", "STC_Replacement", "Repl._STC",
    "NOP/LGS/NRTR/01", "Equipment_ID_NOP/LGS/NRTR/01", "ID_NOP/LGS/NRTR/01", "NRTR/01",
    "NOP/LGS/SIMPC01", "Equipment_ID_NOP/LGS/SIMPC01", "ID_NOP/LGS/SIMPC01", "SIMPC01", "Equipment_ID"
]
# Training examples for Name of Equipment
Eq_Name = [
    "Station_Computer_01","Station_Comp-00","Station_Computer-01_DELL-T5600", "Name_of_Equipment_Station_Computer-01_DELL-T5600", "Station_Computer-01", "DELL-T5600",
    "Station_Computer-03_DELL-T5820", "Name_of_Equipment_Station_Computer-03_DELL-T5820", "Station_Computer-03", "DELL-T5820",
    "GEI_-_SPARE_BECKHOFF/C5101-0010", "Name_of_Equipment_GEI_-_SPARE_BECKHOFF/C5101-0010", "GEI_-_SPARE_BECKHOFF", "C5101-0010",
    "RKVM-1501_RS_400CD_US", "Name_of_Equipment_RKVM-1501_RS_400CD_US", "RKVM-1501", "RS_400CD_US",
    "HP_LaserJet_Printer_P3015", "Name_of_Equipment_HP_LaserJet_Printer_P3015", "HP_LaserJet_Printer", "P3015",
    "NCC_Router-02", "Name_of_Equipment_NCC_Router-02", "NCC_Router", "Router-02",
    "Cisco-2811", "Name_of_Equipment_Cisco-2811", "Cisco-2811", "Cisco",
    "CISCO-1760", "Name_of_Equipment_CISCO-1760", "CISCO-1760", "CISCO",
    "BECKHOFF", "Name_of_Equipment_BECKHOFF", "BECKHOFF",
    "RKVM", "Name_of_Equipment_RKVM", "RKVM",
    "HP_Compaq_pro_6300", "Name_of_Equipment_HP_Compaq_pro_6300", "HP_Compaq_pro_6300", "HP",
    "VHF_Computer", "Name_of_Equipment_VHF_Computer", "VHF_Computer", "VHF",
    "UHF_Computer", "Name_of_Equipment_UHF_Computer", "UHF_Computer", "UHF",
    "DELL_Precision_5820", "Name_of_Equipment_DELL_Precision_5820", "DELL_Precision_5820", "DELL",
    "Ethernet_Switch", "Name_of_Equipment_Ethernet_Switch", "Ethernet_Switch", "Switch",
    "24V/28V_DC_Power_Supply", "Name_of_Equipment_24V/28V_DC_Power_Supply", "24V/28V_DC_Power_Supply", "Power_Supply",
    "Thinvent_Thinclient", "Name_of_Equipment_Thinvent_Thinclient", "Thinvent_Thinclient", "Thinclient",
    "Monitor", "Name_of_Equipment_Monitor", "Monitor",
    "Station_Computer-04_DELL-T5900", "Name_of_Equipment_Station_Computer-04_DELL-T5900", "Station_Computer-04", "DELL-T5900",
    "GEI_-_SPARE_BECKHOFF/C5102-0011", "Name_of_Equipment_GEI_-_SPARE_BECKHOFF/C5102-0011", "GEI_-_SPARE_BECKHOFF", "C5102-0011",
    "RKVM-1502_RS_400CD_US", "Name_of_Equipment_RKVM-1502_RS_400CD_US", "RKVM-1502", "RS_400CD_US",
    "HP_LaserJet_Printer_P3016", "Name_of_Equipment_HP_LaserJet_Printer_P3016", "HP_LaserJet_Printer", "P3016",
    "NCC_Router-03", "Name_of_Equipment_NCC_Router-03", "NCC_Router", "Router-03",
    "Cisco-2812", "Name_of_Equipment_Cisco-2812", "Cisco-2812", "Cisco",
    "CISCO-1761", "Name_of_Equipment_CISCO-1761", "CISCO-1761", "CISCO",
    "BECKHOFF-EtherCAT", "Name_of_Equipment_BECKHOFF-EtherCAT", "BECKHOFF-EtherCAT", "BECKHOFF",
    "RKVM-1503", "Name_of_Equipment_RKVM-1503", "RKVM-1503", "RKVM",
    "HP_Compaq_pro_6301", "Name_of_Equipment_HP_Compaq_pro_6301", "HP_Compaq_pro_6301", "HP",
    "VHF_Radio", "Name_of_Equipment_VHF_Radio", "VHF_Radio", "VHF",
    "UHF_Radio", "Name_of_Equipment_UHF_Radio", "UHF_Radio", "UHF",
    "DELL_Precision_5830", "Name_of_Equipment_DELL_Precision_5830", "DELL_Precision_5830", "DELL",
    "Ethernet_Switch_Pro", "Name_of_Equipment_Ethernet_Switch_Pro", "Ethernet_Switch_Pro", "Switch",
    "24V/28V_AC_Power_Supply", "Name_of_Equipment_24V/28V_AC_Power_Supply", "24V/28V_AC_Power_Supply", "Power_Supply",
    "Thinclient_Ultra", "Name_of_Equipment_Thinclient_Ultra", "Thinclient_Ultra", "Thinclient",
    "Large_Monitor", "Name_of_Equipment_Large_Monitor", "Large_Monitor", "Monitor",
]

# Training examples for Location (Hardware)
Loc_Hardware = [
    "TELEMETRY", "Location_TELEMETRY", "Located_in_TELEMETRY", "TELEMETRY_room",
    "Comp_Lab", "Location_Comp_Lab", "Located_in_Comp_Lab", "Comp_Lab_area",
    "EAST_ANT_ROOM", "Location_EAST_ANT_ROOM", "Located_in_EAST_ANT_ROOM", "EAST_ANT_ROOM_section",
    "WEST_ANT_ROOM", "Location_WEST_ANT_ROOM", "Located_in_WEST_ANT_ROOM", "WEST_ANT_ROOM_zone",
    "SPARE", "Location_SPARE", "Located_in_SPARE", "SPARE_storage",
    "IRDCN", "Location_IRDCN", "Located_in_IRDCN", "IRDCN_area",
    "LAB", "Location_LAB", "Located_in_LAB", "LAB_section",
    "LK3_ANT_ROOM", "Location_LK3_ANT_ROOM", "Located_in_LK3_ANT_ROOM", "LK3_ANT_ROOM",
    "IRDCN_ROOM", "Location_IRDCN_ROOM", "Located_in_IRDCN_ROOM", "IRDCN_ROOM_area",
    "EAST_ANT_ROOM", "Location_EAST_ANT_ROOM", "EAST_ANT_ROOM_location", "East_side_ANT_ROOM",
    "WEST_ANT_ROOM", "Location_WEST_ANT_ROOM", "WEST_ANT_ROOM_place", "West_side_ANT_ROOM",
    "TELEMETRY", "Location_TELEMETRY", "TELEMETRY_location", "TELEMETRY_unit",
    "Comp_Lab", "Location_Comp_Lab", "Comp_Lab_position", "Comp_Lab",
    "LAB", "Location_LAB", "LAB_location", "LAB",
    "SPARE", "Location_SPARE", "SPARE_location", #"SPARE",
    "IRDCN", "Location_IRDCN", "IRDCN_place", "IRDCN_area",
    "LK3_ANT_ROOM", "Location_LK3_ANT_ROOM", "LK3_ANT_ROOM_location", "LK3_ANT_ROOM",
    "EAST_ANT_ROOM", "Location_EAST_ANT_ROOM", "EAST_ANT_ROOM", "EAST_ANT_ROOM",
    "WEST_ANT_ROOM", "Location_WEST_ANT_ROOM", "WEST_ANT_ROOM_area", "WEST_ANT_ROOM",
    "TELEMETRY", "Location_TELEMETRY", "TELEMETRY", "TELEMETRY",
    "Comp_Lab", "Location_Comp_Lab", "Comp_Lab_location", "Comp_Lab",
    "EAST_ANT_ROOM", "Location_EAST_ANT_ROOM", "EAST_ANT_ROOM_section", "EAST_ANT_ROOM",
    "WEST_ANT_ROOM", "Location_WEST_ANT_ROOM", "WEST_ANT_ROOM", "WEST_ANT_ROOM",
    "SPARE", "Location_SPARE", "SPARE_location", "SPARE",
    "IRDCN_ROOM", "Location_IRDCN_ROOM", "IRDCN_ROOM", "IRDCN_ROOM_area",
    "LAB", "Location_LAB", "LAB_area", "LAB",
    "LK3_ANT_ROOM", "Location_LK3_ANT_ROOM", "LK3_ANT_ROOM", "LK3_ANT_ROOM_location",
    "EAST_ANT_ROOM", "Location_EAST_ANT_ROOM", "EAST_ANT_ROOM_zone", "EAST_ANT_ROOM",
    "WEST_ANT_ROOM", "Location_WEST_ANT_ROOM", "WEST_ANT_ROOM", "WEST_ANT_ROOM",
    "TELEMETRY", "Location_TELEMETRY", "TELEMETRY", "TELEMETRY",
    "Comp_Lab", "Location_Comp_Lab", "Comp_Lab", "Comp_Lab",
    "IRDCN", "Location_IRDCN", "IRDCN", "IRDCN",
    "SPARE", "Location_SPARE", "SPARE", "SPARE", #"Located", "Location", "Area", "Storage",
    "LAB", "Location_LAB", "LAB", "LAB",
    "LK3_ANT_ROOM", "Location_LK3_ANT_ROOM", "LK3_ANT_ROOM", "LK3_ANT_ROOM"
]
# Training examples for Serial No.
S_No = [
    "C1954Y1", "Serial_No._C1954Y1", "C1954Y1_serial_number", "Serial_number_C1954Y1",
    "81954Y1", "Serial_No._81954Y1", "81954Y1_serial", "Serial_number_81954Y1",
    "FF2ZQ13", "Serial_No._FF2ZQ13", "FF2ZQ13_number", "Serial_number_FF2ZQ13",
    "712656.035", "Serial_No._712656.035", "712656.035_serial", "Serial_number_712656.035",
    "080104034", "Serial_No._080104034", "080104034_serial", "Serial_number_080104034",
    "VNB8B5YJ2G", "Serial_No._VNB8B5YJ2G", "VNB8B5YJ2G_number", "Serial_number_VNB8B5YJ2G",
    "FMKD85011UG", "Serial_No._FMKD85011UG", "FMKD85011UG_serial", "Serial_number_FMKD85011UG",
    "5DR1VG2", "Serial_No._5DR1VG2", "5DR1VG2_number", "Serial_number_5DR1VG2",
    "22954Y1", "Serial_No._22954Y1", "22954Y1_serial_number", "Serial_number_22954Y1",
    "INA308X24M", "Serial_No._INA308X24M", "INA308X24M_number", "Serial_number_INA308X24M",
    "7XY2V72", "Serial_No._7XY2V72", "7XY2V72_serial", "Serial_number_7XY2V72",
    "BCV2241J0B1", "Serial_No._BCV2241J0B1", "BCV2241J0B1_number", "Serial_number_BCV2241J0B1",
    "5DRIVG2", "Serial_No._5DRIVG2", "5DRIVG2_serial", "Serial_number_5DRIVG2",
    "C1954Y1", "Serial_No._C1954Y1", "C1954Y1", "Serial_number_C1954Y1",
    "81954Y1", "Serial_No._81954Y1", "81954Y1", "Serial_number_81954Y1",
    "FF2ZQ13", "Serial_No._FF2ZQ13", "FF2ZQ13", "Serial_number_FF2ZQ13",
    "712656.035", "Serial_No._712656.035", "712656.035", "Serial_number_712656.035",
    "080104034", "Serial_No._080104034", "080104034", "Serial_number_080104034",
    "VNB8B5YJ2G", "Serial_No._VNB8B5YJ2G", "VNB8B5YJ2G", "Serial_number_VNB8B5YJ2G",
    "FMKD85011UG", "Serial_No._FMKD85011UG", "FMKD85011UG", "Serial_number_FMKD85011UG",
    "5DR1VG2", "Serial_No._5DR1VG2", "5DR1VG2", "Serial_number_5DR1VG2",
    "22954Y1", "Serial_No._22954Y1", "22954Y1", "Serial_number_22954Y1",
    "INA308X24M", "Serial_No._INA308X24M", "INA308X24M", "Serial_number_INA308X24M",
    "7XY2V72", "Serial_No._7XY2V72", "7XY2V72", "Serial_number_7XY2V72",
    "BCV2241J0B1", "Serial_No._BCV2241J0B1", "BCV2241J0B1", "Serial_number_BCV2241J0B1",
    "5DRIVG2", "Serial_No._5DRIVG2", "5DRIVG2", "Serial_number_5DRIVG2",
    "C1954Y1", "Serial_No._C1954Y1", "C1954Y1", "Serial_number_C1954Y1",
    "81954Y1", "Serial_No._81954Y1", "81954Y1", "Serial_number_81954Y1",
    "FF2ZQ13", "Serial_No._FF2ZQ13", "FF2ZQ13", "Serial_number_FF2ZQ13",
    "712656.035", "Serial_No._712656.035", "712656.035", "Serial_number_712656.035",
    "080104034", "Serial_No._080104034", "080104034", "Serial_number_080104034",
    "VNB8B5YJ2G", "Serial_No._VNB8B5YJ2G", "VNB8B5YJ2G", "Serial_number_VNB8B5YJ2G",
    "FMKD85011UG", "Serial_No._FMKD85011UG", "FMKD85011UG", "Serial_number_FMKD85011UG",
    "5DR1VG2", "Serial_No._5DR1VG2", "5DR1VG2", "Serial_number_5DR1VG2",
    "22954Y1", "Serial_No._22954Y1", "22954Y1", "Serial_number_22954Y1",
    "INA308X24M", "Serial_No._INA308X24M", "INA308X24M", "Serial_number_INA308X24M",
    "7XY2V72", "Serial_No._7XY2V72", "7XY2V72", "Serial_number_7XY2V72",
    "BCV2241J0B1", "Serial_No._BCV2241J0B1", "BCV2241J0B1", "Serial_number_BCV2241J0B1",
    "5DRIVG2", "Serial_No._5DRIVG2", "5DRIVG2", "Serial_number_5DRIVG2"
]

# Training examples for Store Ref. No.
SR_No = [
    "ISTRAC/A00020061844", "Store_Ref._No._ISTRAC/A00020061844", "ISTRAC/A00020061844_ref", "Ref._ISTRAC/A00020061844",
    "ISTRAC/A00020061843", "Store_Ref._No._ISTRAC/A00020061843", "ISTRAC/A00020061843_ref", "Ref._ISTRAC/A00020061843",
    "ISTRAC/A00020070524", "Store_Ref._No._ISTRAC/A00020070524", "ISTRAC/A00020070524_ref", "Ref._ISTRAC/A00020070524",
    "ISTRAC/A00020061850", "Store_Ref._No._ISTRAC/A00020061850", "ISTRAC/A00020061850_ref", "Ref._ISTRAC/A00020061850",
    "ISTRAC/IE/5334", "Store_Ref._No._ISTRAC/IE/5334", "ISTRAC/IE/5334_ref", "Ref._ISTRAC/IE/5334",
    "ISTRAC/IE/5336", "Store_Ref._No._ISTRAC/IE/5336", "ISTRAC/IE/5336_ref", "Ref._ISTRAC/IE/5336",
    "ISTRAC/A00020061844", "Ref._ISTRAC/A00020061844", "Store_ISTRAC/A00020061844", "Number_ISTRAC/A00020061844",
    "ISTRAC/A00020061843", "Ref._ISTRAC/A00020061843", "Store_ISTRAC/A00020061843", "Number_ISTRAC/A00020061843",
    "ISTRAC/A00020070524", "Ref._ISTRAC/A00020070524", "Store_ISTRAC/A00020070524", "Number_ISTRAC/A00020070524",
    "ISTRAC/A00020061850", "Ref._ISTRAC/A00020061850", "Store_ISTRAC/A00020061850", "Number_ISTRAC/A00020061850",
    "ISTRAC/IE/5334", "Ref._ISTRAC/IE/5334", "Store_ISTRAC/IE/5334", "Number_ISTRAC/IE/5334",
    "ISTRAC/IE/5336", "Ref._ISTRAC/IE/5336", "Store_ISTRAC/IE/5336", "Number_ISTRAC/IE/5336",
    "Store_Ref._ISTRAC/A00020061844", "Store_Ref._ISTRAC/A00020061843", "Store_Ref._ISTRAC/A00020070524",
    "Store_Ref._ISTRAC/A00020061850", "Store_Ref._ISTRAC/IE/5334", "Store_Ref._ISTRAC/IE/5336",
    "Store_ISTRAC/A00020061844", "Store_ISTRAC/A00020061843", "Store_ISTRAC/A00020070524",
    "Store_ISTRAC/A00020061850", "Store_ISTRAC/IE/5334", "Store_ISTRAC/IE/5336",
    "Ref._ISTRAC/A00020061844", "Ref._ISTRAC/A00020061843", "Ref._ISTRAC/A00020070524",
    "Ref._ISTRAC/A00020061850", "Ref._ISTRAC/IE/5334", "Ref._ISTRAC/IE/5336",
    "Number_ISTRAC/A00020061844", "Number_ISTRAC/A00020061843", "Number_ISTRAC/A00020070524",
    "Number_ISTRAC/A00020061850", "Number_ISTRAC/IE/5334", "Number_ISTRAC/IE/5336"
]

# Training examples for Asset ID
Asset_ID =[
    "A00020130592", "A00020130607", "A00020070496", "A00020200422", "A00020191234",
    "A00020210011", "A00020231456", "A00020252000", "A00020031234", "A00020141122",
    "A00020152233", "A00020214567", "A00020163344", "A00020225478", "A00020071123",
    "A00020172234", "A00020231011", "A00020183200", "A00020091234", "A00020194455",
    "A00020202345", "A00020212567", "A00020105678", "A00020219876", "A00020081234",
    "A00020113344", "A00020124455", "A00020232200", "A00020090112", "A00020134567",
    "A00020243321", "A00020155678", "A00020251010", "A00020260000", "A00020166789",
    "A00020271111", "A00020070809", "A00020284456", "A00020177890", "A00020188899",
    "A00020295555", "A00020100001", "A00020111112", "A00020090001", "A00020212223",
    "A00020223334", "A00020134555", "A00020245566", "A00020056789", "A00020167890",
    "A00020278899", "A00020178901", "A00020289900", "A00020189012", "A00020291234",
    "A00020200123", "A00020011022", "A00020122334", "A00020233445", "A00020045678",
    "Asset_ID_A00020130592", "Asset_ID_A00020130607", "Asset_ID_A00020070496",
    "Asset_ID_A00020200422", "Asset_ID_A00020191234", "Asset_ID_A00020210011",
    "Asset_ID_A00020231456", "Asset_ID_A00020252000", "Asset_ID_A00020031234",
    "Asset_ID_A00020141122", "Asset_ID_A00020152233", "Asset_ID_A00020214567",
    "Asset_ID_A00020163344", "Asset_ID_A00020225478", "Asset_ID_A00020071123",
    "Asset_ID_A00020172234", "Asset_ID_A00020231011", "Asset_ID_A00020183200",
    "Asset_ID_A00020091234", "Asset_ID_A00020194455", "Asset_ID_A00020202345"
]

# Training examples for Column Name
# Col_Name = [
#     "Column_Name_Equipment_ID", "Column_Name_Name_of_Equipment", "Column_Name_Location",
#     "Column_Name_Serial_No.", "Column_Name_Store_Ref._No.", "Column_Name_Asset_ID",
#     "Column_Name_Software_ID", "Column_Name_Software_Name", "Column_Name_Size",
#     "Column_Name_Location_(Software)", "Column_Name_Revision_No.", "Column_Name_Date_of_Updation",
#     "Column_Name_Action", "Column_Name_General", "Column_Name", "Column_Name_Equipment_ID",
#     "Column_Name_Name_of_Equipment", "Column_Name_Location_(Hardware)", "Column_Name_Serial_No.",
#     "Column_Name_Store_Ref._No.", "Column_Name_Asset_ID", "Column_Name_Software_ID",
#     "Column_Name_Software_Name", "Column_Name_Size", "Column_Name_Location_(Software)",
#     "Column_Name_Revision_No.", "Column_Name_Date_of_Updation", "Column_Name_Action",
#     "Column_Name_General", "Column_Name_Equipment_ID", "Column_Name_Name_of_Equipment",
#     "Column_Name_Location", "Column_Name_Serial_No.", "Column_Name_Store_Ref._No.",
#     "Column_Name_Asset_ID", "Column_Name_Software_ID", "Column_Name_Software_Name",
#     "Column_Name_Size", "Column_Name_Location_(Software)", "Column_Name_Revision_No.",
#     "Column_Name_Date_of_Updation", "Column_Name_Action", "Column_Name_General",
#     "Column_Name_Equipment_ID", "Column_Name_Name_of_Equipment", "Column_Name_Location",
#     "Column_Name_Serial_No.", "Column_Name_Store_Ref._No.", "Column_Name_Asset_ID",
#     "Column_Name_Software_ID", "Column_Name_Software_Name", "Column_Name_Size",
#     "Column_Name_Location_(Software)", "Column_Name_Revision_No.", "Column_Name_Date_of_Updation",
#     "Column_Name_Action", "Column_Name_General", "Column_Name_Equipment_ID",
#     "Column_Name_Name_of_Equipment", "Equipment_ID", "Name_of_Equipment", "Location", "Serial_No.",
#     "Store_Ref._No.", "Asset_ID", "Software_ID", "Software_Name",
#     "Size", "Location_(Software)", "Revision_No.", "Date_of_Updation",
#     "Action", "General", "Equipment_ID", "Name_of_Equipment",
#     "Location_(Hardware)", "Serial_No.", "Store_Ref._No.",
#     "Asset_ID", "Software_ID", "Software_Name", "Size",
#     "Location_(Software)", "Revision_No.", "Date_of_Updation",
#     "Action", "General", "Equipment_ID", "Name_of_Equipment",
#     "Location", "Serial_No.", "Store_Ref._No.", "Asset_ID",
#     "Software_ID", "Software_Name", "Size", "Location_(Software)",
#     "Revision_No.", "Date_of_Updation", "Action", "General",
#     "Equipment_ID", "Name_of_Equipment", "Location",
#     "Serial_No.", "Store_Ref._No.", "Asset_ID", "Software_ID",
#     "Software_Name", "Size", "Location_(Software)", "Revision_No.",
#     "Date_of_Updation", "Action", "General", "Equipment_ID", "Name_of_Equip", "Location_Info", "Serial_No",
#     "Store_Ref_Number", "Asset_Identifier", "Software_ID", "Software_Title",
#     "Software_Size", "Software_Location", "Revision_Number", "Update_Date",
#     "Action_Taken", "General_Info", "Equip_ID", "Name_of_Equipment",
#     "Location_(Hardware)", "Serial_Number", "Store_Reference_No.",
#     "Asset_Identification", "Software_Code", "Software_Name", "Software_Size",
#     "Location_of_Software", "Revision_Information", "Date_Updated",
#     "Action_Required", "General_Details", "Equipment_ID", "Equipment_Name",
#     "Location_(H/W)", "Serial_Number", "Store_Ref_No.", "Asset_Code",
#     "Software_ID", "Software_Description", "Software_Size", "Software_Location",
#     "Revision_Info", "Date_of_Update", "Action_Taken", "General_Info",
#     "Equipment_ID", "Equipment_Details", "Location", "Serial_Number",
#     "Store_Ref._No.", "Asset_ID", "Software_ID", "Software_Title",
#     "Software_Size", "Software_Location", "Revision_No.", "Update_Date"
# ]

# Training examples for General token classification
General = [
    "Show", "List", "Fetch", "Retrieve", "Display", "Find",
    "Get", "Show me", "List all", "Provide", "Show details",
    "Show me the", "List the", "Fetch details", "Get information",
    "Provide details", "Find details", "Retrieve data", "Show information",
    "Display details", "Display all", "Get list", "Show records",
    "Retrieve list", "Find all", "Get data", "List records",
    "Provide records", "Fetch records", "Show data", "Find records",
    "Show items", "Retrieve items", "Display items", "List items",
    "Get items", "Show info", "Fetch info", "Find info",
    "Display info", "List info", "Provide info", "Show entry",
    "Retrieve entry", "Display entry", "List entry", "Get entry",
    "Find entry", "Show output", "Fetch output", "Retrieve output",
    "Display output", "List output", "Get output", "Find output",
    "Show list", "Retrieve list", "Display list", "Fetch list",
    "List info", "Show data", "Get details", "Display records",
    "Find data", "Retrieve records", "Show complete", "Fetch complete",
    "List complete", "Get complete", "Find complete", "Show overview",
    "Retrieve overview", "Display overview", "List overview", "Get overview",
    "Find overview", "Show summary", "Fetch summary", "Retrieve summary",
    "Display summary", "List summary", "Get summary", "Find summary",
    "Show all", "Retrieve all", "Display all", "List all", "Get all",
    "each", "every", "all", "both", "either", "neither", "one", "none",
    "some", "any", "much", "more", "most", "few", "several", "couple",
    "every", "every", "no", "nothing", "everything", "something", "anything",
    "each", "every", "many", "few", "several", "another", "last", "next",
    "first", "last", "second", "third", "few", "more", "most", "less",
    "most", "least", "every", "every", "where", "somewhere", "anywhere",
    "nowhere", "here", "there", "near", "far", "inside", "outside", "above",
    "below", "underneath", "over", "beneath", "between", "among", "within",
    "around", "throughout", "beside", "alongside", "toward", "backward",
    "forward", "upward", "downward", "side", "left", "right", "straight",
    "beyond", "close", "close", "away", "inside", "outside", "along",
    "against", "towards", "amongst", "amidst", "afterwards", "beforehand",
    "hereafter", "thereafter", "earlier", "later", "above", "below",
    "up", "down", "under", "over", "underneath", "beneath", "over",
    "under", "between", "among", "between", "toward", "from", "by",
    "upon", "according", "except", "outside", "beside", "along",
    "through", "until", "during", "before", "after", "while", "if",
    "unless", "as", "unless", "until", "since", "until", "once",
    "if", "provided", "unless", "even", "though", "although", "unless",
    "whether", "as", "like", "than", "when", "where", "how", "why",
    "what", "which", "who", "whom", "whose", "that", "this", "these",
    "those", "another", "other", "some", "any", "much", "more",
    "many", "few", "several", "both", "either", "neither", "such",
    "same", "different", "better", "worse", "more", "less", "least",
    "most", "more", "few", "fewer", "less", "last", "next", "recent",
    "old", "young", "new", "early", "late", "recent", "past", "future",
    "a", "an", "the", "in", "on", "at", "by", "with", "of", "for",
    "to", "from", "about", "under", "over", "between", "among", "near",
    "through", "within", "around", "beside", "behind", "after", "before",
    "during", "past", "since", "until", "as", "like", "so", "but",
    "or", "and", "nor", "if", "then", "else", "when", "where", "how",
    "why", "which", "what", "who", "whom", "whose", "that", "this",
    "these", "those", "one", "ones", "some", "any", "each", "every",
    "all", "many", "few", "several", "more", "most", "less", "least",
    "much", "just", "only", "also", "very", "such", "as", "than", "than",
    "both", "each", "either", "neither", "another", "few", "several",
    "many", "much", "one", "each", "every", "some", "any", "more",
    "most", "less", "least", "few", "little", "several", "several",
    "couple", "do", "does", "did", "has", "have", "had", "will",
    "shall", "would", "could", "should", "might", "may", "can", "must",
    "need", "ought", "like", "seem", "seems", "appears", "show", "make",
    "keep", "come", "go", "get", "give", "take", "bring", "send",
    "reply", "return", "make", "make", "allow", "ask", "want",
    "need", "prefer", "try", "start", "stop", "finish", "continue", "turn",
    "change", "move", "stay", "remain", "stay", "keep", "hold", "wait",
    "look", "see", "hear", "find", "find", "lose", "search", "check",
    "find", "search", "show", "find", "find", "describe", "explain",
    "give", "provide", "offer", "request", "query", "ask", "answer",
    "resolve", "solve", "complete", "complete", "fix", "solve", "resolve",
    "show", "find", "give", "provide", "offer", "describe", "explain",
    "request", "query", "ask", "answer", "resolve", "solve", "complete",
    '?','!','.'
]

# Creating training sets
x_train_tokens = []
x_train_tokens.extend(Eq_ID)
x_train_tokens.extend(Eq_Name)
x_train_tokens.extend(Loc_Hardware)
x_train_tokens.extend(S_No)
x_train_tokens.extend(SR_No)
x_train_tokens.extend(Asset_ID)
# x_train_tokens.extend(Col_Name)
x_train_tokens.extend(General)

# Corresponding labels
y_train_tokens = ["Equipment ID"] * len(Eq_ID) + ["Name of Equipment"] * len(Eq_Name) + ["Location (Hardware)"] * len(Loc_Hardware) + ["Serial No."] * len(S_No) + ["Store Ref. No."] * len(SR_No) + ["Asset ID"] * len(Asset_ID) + ["General"] * len(General) #+ ["Column Name"] * len(Col_Name)

print (len(y_train_tokens))
print (len(x_train_tokens))

1141
1141


In [ ]:
# Generating token embeddings for training
x_train_token_embeddings = [preprocess_query(token).mean(dim=1).detach().numpy()[0] for token in x_train_tokens]

# Train the token classification model
token_classifier = RandomForestClassifier()
token_classifier.fit(x_train_token_embeddings, y_train_tokens)

RandomForestClassifier()

In [ ]:
# # Loading spacy for nlp operations
# nlp = spacy.load("en_core_web_sm")

# # Training the token classification data
# token_data = {"token": x_train_tokens, "label": y_train_tokens}
# # Convert to DataFrame
# df_tokens = pd.DataFrame(token_data)

# nlp = spacy.load("en_core_web_sm")

# # Function to extract features from a token
# def extract_features(token):
#     doc = nlp(token)
#     features = {
#         "text": token,
#         "pos": doc[0].pos_,
#         "dep": doc[0].dep_
#     }
#     return features

# # Extract features for each token
# features = [extract_features(row["token"]) for _, row in df_tokens.iterrows()]

# # Convert features to DataFrame
# df_features = pd.DataFrame(features)

# # Vectorize the features
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(df_features["text"])
# y = df_tokens["label"]

# # Train RandomForestClassifier
# clf = RandomForestClassifier()
# clf.fit(X, y)

In [ ]:
class CustomTokenizer:
    def __init__(self, multi_word_tokens):
        self.multi_word_tokens = sorted(multi_word_tokens, key=len, reverse=True)  # Sort by length for longest match first
        self.token_pattern = re.compile(r'\b(?:' + '|'.join(re.escape(token) for token in self.multi_word_tokens) + r')\b')

    def tokenize(self, text):
        tokens = []
        last_index = 0

        for match in self.token_pattern.finditer(text):
            start, end = match.span()
            if start > last_index:
                non_matching_text = text[last_index:start].split()
                tokens.extend(non_matching_text)
            tokens.append(match.group())
            last_index = end

        if last_index < len(text):
            tokens.extend(text[last_index:].split())

        return tokens

# Example multi-word tokens
data01 = pd.read_excel('multiwordlist.xlsx')
df01 = pd.DataFrame(data01)
multi_word_tokens = df01.iloc[:, 0].tolist()
print(multi_word_tokens)
tokenizer = CustomTokenizer(multi_word_tokens)


['Replacement for STC ', 'Replacement STC ', 'STC Replacement ', 'Station Computer-01 DELL-T5600', 'Station Computer-01 DELL-T5600', 'Station Computer-01 ', 'Station Computer-03 DELL-T5820 ', 'Station Computer-03 ', 'GEI - SPARE BECKHOFF/C5101-0010 ', 'GEI - SPARE BECKHOFF', 'RKVM-1501 RS_400CD_US', 'HP LaserJet Printer P3015', 'HP LaserJet Printer ', 'NCC Router-02 ', 'NCC Router ', 'HP Compaq pro 6300', 'VHF Computer ', 'UHF Computer ', 'DELL Precision 5820', 'Ethernet Switch', '24V/28V DC Power Supply', 'Power Supply', 'Thinvent Thinclient ', 'Station Computer-04 DELL-T5900', 'Station Computer-04 ', 'GEI - SPARE BECKHOFF/C5102-0011 ', 'GEI - SPARE BECKHOFF', 'RKVM-1502 RS_400CD_US', 'HP LaserJet Printer P3016 ', 'HP LaserJet Printer', 'NCC Router-03', 'NCC Router', 'HP Compaq pro 6301 ', 'HP Compaq pro 6301 ', 'VHF Radio', 'UHF Radio', 'DELL Precision 5830 ', 'Ethernet Switch Pro', 'Comp Lab ', 'EAST ANT ROOM ', 'WEST ANT ROOM ', 'SPARE', 'IRDCN ROOM ', 'Computer Lab ', 'IRDCN ROOM 

In [ ]:
# # Load spaCy's pre-trained model for NER
# nlp = spacy.load("en_core_web_sm")

# class CustomDynamicTokenizer:
#     def __init__(self, nlp):
#         self.nlp = nlp

#     def tokenize(self, text):
#         doc = self.nlp(text)
#         tokens = []
#         for ent in doc.ents:
#             tokens.append(ent.text)
#         for token in doc:
#             if not token.ent_type_:
#                 tokens.append(token.text)
#         return tokens

# # Example usage
# tokenizer = CustomDynamicTokenizer(nlp)

In [ ]:
query = "Where is stn_comp-01 located?"

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
embedding_model = AutoModel.from_pretrained('bert-base-uncased')

# Preprocess query to get embeddings
def preprocessing_query(query):
    inputs = bert_tokenizer(query, return_tensors='pt')
    outputs = embedding_model(**inputs)
    return outputs.last_hidden_state

# Function to classify tokens
def classify_tokens(query):
    # Tokenize using the custom tokenizer
    tokens = tokenizer.tokenize(query)

    # Generate embeddings for each token using BERT
    token_embeddings = []
    for token in tokens:
        embedding = preprocessing_query(token).mean(dim=1).detach().numpy()[0]
        token_embeddings.append(embedding)

    # Convert token_embeddings to a numpy array and reshape if necessary
    token_embeddings = np.array(token_embeddings)
    if len(token_embeddings.shape) == 1:
        token_embeddings = token_embeddings.reshape(1, -1)

    # Classify tokens using the trained model
    classifications = token_classifier.predict(token_embeddings)

    return list(zip(tokens, classifications))

# Test the classifier with a query
# query = "Where is equipment name NOP/LGS/STC/02, located in Stn_Comp-01 DELL-T5600, for HP_Compaq_pro_6300, Power_Supply, f88ss6y40 and Beckhoff-ethercat?"
classified_tokens = classify_tokens(query)

# Display results
print("Classified Tokens:")
for token, label in classified_tokens:
    print(f"Token: {token}, Label: {label}")

Classified Tokens:
Token: Where, Label: General
Token: is, Label: General
Token: stn_comp-01, Label: Name of Equipment
Token: located?, Label: General


In [ ]:
# # Load spaCy's pre-trained NER model
# nlp = spacy.load("en_core_web_sm")

# def apply_ner_to_classified_tokens(classified_tokens):
#     ner_results = []
#     for token, label in classified_tokens:
#         if label in ['Name of Equipment', 'Equipment ID']:  # Apply NER only to relevant labels
#             doc = nlp(token)
#             entities = [(ent.text, ent.label_) for ent in doc.ents]
#             ner_results.append((token, label, entities))
#         else:
#             ner_results.append((token, label, []))
#     return ner_results

# # Example usage
# ner_results = apply_ner_to_classified_tokens(classified_tokens)

# # Display results
# for token, label, entities in ner_results:
#     print(f"Token: {token}, Label: {label}, Entities: {entities}")


In [ ]:
# !pip install contractions

In [ ]:
# import contractions

# def expand_contractions(text):
#     return contractions.fix(text)

# # Example usage
# text = "I can't go to stn. Cuz my comp is fried."
# expanded_text = expand_contractions(text)
# print(expanded_text)

In [ ]:
# import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# from nltk.corpus import wordnet
# from nltk.tokenize import word_tokenize

# def get_synonyms(word):
#     synonyms = set()
#     for syn in wordnet.synsets(word):
#         for lemma in syn.lemmas():
#             synonyms.add(lemma.name())
#     return synonyms

# def standardize_synonyms(text):
#     tokens = word_tokenize(text)
#     standardized_tokens = []
#     for token in tokens:
#         synonyms = get_synonyms(token)
#         if synonyms:
#             # Choose the first synonym (you might want a better strategy)
#             standardized_tokens.append(list(synonyms)[0])
#         else:
#             standardized_tokens.append(token)
#     return ' '.join(standardized_tokens)

# # Example usage
# text = "Find the asset number for the equip ID"
# standardized_text = standardize_synonyms(text)
# print(standardized_text)

In [ ]:
# import openai

# # Function to call GPT-3/4 for intention recognition and token classification
# def analyze_query(query):
#     # Initial prompt with examples
#     prompt = """
#     You are an AI designed to interpret natural language queries for database searches. Your task is to:
#     1. Identify the table to be searched.
#     2. Classify tokens in the query to identify relevant columns.
#     3. Normalize the query for consistent terminology.

#     Here are some examples:

#     Query: "Where is equipment name A00020210098 located?"
#     - Table: equipment
#     - Columns: [name, location]
#     - Normalized Query: "SELECT location FROM equipment WHERE name = 'A00020210098'"

#     Query: "Show me the software details for ID SFT12345."
#     - Table: software
#     - Columns: [ID, details]
#     - Normalized Query: "SELECT details FROM software WHERE ID = 'SFT12345'"

#     Now, analyze the following query and provide the table, columns, and normalized query:
#     """

#     # Add the user's query to the prompt
#     prompt += f"Query: \"{query}\"\n"

#     # Call the OpenAI API
#     response = openai.Completion.create(
#         engine="text-davinci-003",  # or the latest model available
#         prompt=prompt,
#         max_tokens=150,
#         n=1,
#         stop=None,
#         temperature=0.5,
#     )

#     # Parse the response
#     output = response.choices[0].text.strip()
#     return output

# # Example usage
# query = "Where is equipment name A00020210098 located?"
# result = analyze_query(query)
# print(result)

In [ ]:
# !pip install transformers

In [ ]:
# from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [ ]:
# model_name = 'gpt2-large'#"gpt2"  # or 'gpt2-medium', 'gpt2-large', etc.
# tokenizerX = GPT2Tokenizer.from_pretrained(model_name)
# modelX = GPT2LMHeadModel.from_pretrained(model_name)


In [ ]:
# initial_promp = "You are an AI model trained to generate synonyms and abbreviations for a given word. If a text is given parse the text into individual words and output the abbreviations and synonyms. The word is: "

In [ ]:
# User input query
# user_query = "Station Computer 01"

# Combine initial prompt and user input
# input_text = initial_prompt

# # Tokenize input text
# input_ids = tokenizerX.encode(input_text, return_tensors='pt')

# # Generate output
# output = modelX.generate(input_ids, max_length=1000, num_return_sequences=1, no_repeat_ngram_size=2)

# # Decode and print output
# normalized_query = tokenizerX.decode(output[0], skip_special_tokens=True)
# print("Normalized Query:", normalized_query)


In [ ]:
from google.colab import userdata

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('gemini')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash


In [ ]:
modelY = genai.GenerativeModel('gemini-1.5-pro')

In [ ]:
def mapping (token_word):
    mappings = ["Station_Computer_01", "Equipment_ID", "Equipment", "Telemetry", "Station_Computer_02", "EAST_ANT_ROOM", "28V/24V_DC_Power_Supply", "West_Ant_Room"]
    ambiguous_word = token_word
    mapping_prompt = f"""
      You are an AI that helps to find suitable mappings for ambiguous words in database queries.
      Given a list of possible mappings, identify the most appropriate mapping for the provided ambiguous word. The ambiguous word may be an abbreviation or may include synonym of the mapping word.
      The list of possible mappings is as follows:
      {mappings}

      For example:
      Ambiguous Word: "equip"
      Suitable Mapping: "equipment"

      Ambiguous Word: "Telemet"
      Suitable Mapping: "Telemetry"

      Find the suitable mapping for:
      Ambiguous Word: "{ambiguous_word}"
      Suitable Mapping:

      Just give the  suitable mapping word(s) (separated by "and" if more than one). No need of explanation.
      If no suitable or misleading mapping found output text: "No suitable mapping found".
      """
    # %%time
    response = modelY.generate_content(mapping_prompt, stream=True)

    # for chunk in response:
    #   print(chunk.text)

    # Initialize an empty string to hold the complete response
    complete_response = ""

    # Concatenate all chunks into the complete response string
    for chunk in response:
        complete_response += chunk.text

    # Print the complete response
    print(complete_response)
    if(complete_response == "No suitable mapping found"):
      return ambiguous_word
    else:
      return complete_response

In [ ]:
re_query = "";
label_token = {}
for token, label in classified_tokens:
    if (label != "General"):
        new_token = mapping(token)
        # print(token)
        label_token.update({label:new_token})
        re_query = re_query + " " + new_token
    else:
        re_query = re_query + " " + token
print(re_query)

Station_Computer_01 

 Where is Station_Computer_01 
 located?


In [ ]:
table = "Equipment"
# intent, confidence = recognize_intent(re_query, intent_model)
# print(f"Intent recognized: {intent}, Confidence level: {confidence:.2f}")
# if (intent == 0):
#     table = "Equipment"
# else:
#     table = "Software"

In [ ]:
modelX = genai.GenerativeModel('gemini-1.5-pro-latest')

In [ ]:
query_generator_prompt = f"""
    You are an AI that generates SQL queries from a given natural language query.
    The table data structure is as follows:
    Table-1: Equipments Table -
    Column-1: Equipment ID
    Column-2: Equipment Name
    Column-3: Location (Hardware)
    Column-4: Serial No.
    Column-5: Store Ref. No.
    Column-6: Asset ID

    Some guidelines to form the query are:
    1. SQL Query structure must start with SELECT.
    2. The natural language query has words separated by spaces.
    3. The natural language query, table name and the possible key value pairs of column_name and specific_records will be provided to you.

    Provided information:
    1. Natural Language Query: {re_query}
    2. Table name: {table}
    3. Column name : Specific_records :: {label_token}

    Give the natural language query and the SQL query only. No explanation needed.
    """

In [ ]:
%%time
response = modelX.generate_content(query_generator_prompt, stream=True)

# for chunk in response:
#   print(chunk.text)

# Initialize an empty string to hold the complete response
complete_response = ""

# Concatenate all chunks into the complete response string
for chunk in response:
    complete_response += chunk.text

# Print the complete response
print(complete_response)

Natural Language Query: Where is Station_Computer_01 located? 
SQL Query: SELECT Location FROM Equipment WHERE `Name of Equipment` = 'Station_Computer_01'
CPU times: user 75.4 ms, sys: 5.2 ms, total: 80.6 ms
Wall time: 4.52 s
